# 詞表

In [1]:
with open("data/worddata.txt", "rt",encoding="utf-8") as fin:
    lines = fin.readlines()
lines[0] = lines[0].replace("\ufeff","")

In [2]:
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
        return True
    else:
        return False
word_dic={}
for l in lines:
    wordlist = l.split("&")
    if len(wordlist)!=4:
        continue
    for w in list(wordlist[0].replace(" ","")):
        if is_chinese(w)== False:
            continue
        if w in word_dic.keys():
            word_dic[w]+= int(wordlist[2])
        else:
            word_dic[w] = int(wordlist[2])

In [3]:
with open("data/firstword.txt", "wt",encoding="utf-8") as dic:
    sorted_word_dict = sorted(word_dic.items(), key=lambda kv: kv[1], reverse=True)
    for w in sorted_word_dict:
        dic.write(w[0]+"\n")

In [4]:
with open("data/firstword.txt", "rt",encoding="utf-8") as fin:
    txt = fin.read()
firstword = txt.split("\n")
del firstword[-1]

# 錯誤實例

In [5]:
from bs4 import BeautifulSoup 
with open("data/wrong1.txt", "rt",encoding="utf-8") as fin:
    txt = fin.read()
soup = BeautifulSoup(txt, "lxml")
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
        return True
    else:
        return False

In [6]:
WrongWord = {}
for w in firstword:
    WrongWord[w] = {}

for essay in soup.select("essay"):
    mistake = essay.select("mistake")
    for m in mistake:
        # 錯字行字數不等於正確行，這是多或少某字而非錯字，所以跳過
        if len(m.select("wrong")[0].text)!= len(m.select("correction")[0].text): 
            continue
        location = int(m["location"])-1
        wrong = essay.find(id = m["id"]).text[location]
        wrong_index = m.select("wrong")[0].text.index(wrong)
        correct = m.select("correction")[0].text[wrong_index]
        if wrong == correct:
            continue
        if is_chinese(wrong) == False or is_chinese(correct) == False: # 非中文字跳過
            continue
      
        if correct not in WrongWord.keys(): #不在4743字的不收
            continue
        else:
            if wrong not in WrongWord[correct].keys(): 
                WrongWord[correct][wrong] = []
                WrongWord[correct][wrong].append(1)
                try:
                    WrongWord[correct][wrong].append((firstword.index(wrong)+1)*-1) # 加入wrong在firstword的index，但因為排序問題要先加1(為了0)乘-1
                except:
                    WrongWord[correct][wrong].append(-4745)#wrong 不在firstword裡的wrong統一加在最後
            else:
                WrongWord[correct][wrong][0]+=1

In [7]:
with open("data/wrong2.txt", "rt",encoding="utf-8") as fin:
    txt = fin.read()
soup = BeautifulSoup(txt, "lxml")

In [8]:
for doc in soup.select("doc"):
    mistake = doc.select("mistake")
    for m in mistake:
        # 錯字行字數不等於正確行，這是多或少某字而非錯字，所以跳過
        if len(m.select("wrong")[0].text)!= len(m.select("correct")[0].text): 
            continue
        location = int(m["wrong_position"])-1
        wrong = doc.select("p")[0].text[location]
        wrong_index = m.select("wrong")[0].text.index(wrong)
        correct = m.select("correct")[0].text[wrong_index]
        if wrong == correct:
            continue
            print(wrong)
        if is_chinese(wrong) == False or is_chinese(correct) == False: # 非中文字跳過
            continue
      
        if correct not in WrongWord.keys(): #不在4743字的不收
            continue
        else:
            if wrong not in WrongWord[correct].keys(): 
                WrongWord[correct][wrong] = []
                WrongWord[correct][wrong].append(1)
                try:
                    WrongWord[correct][wrong].append((firstword.index(wrong)+1)*-1) # 加入wrong在firstword的index，但因為排序問題要先加1(為了0)乘-1
                except:
                    WrongWord[correct][wrong].append(-4745)#wrong 不在firstword裡的wrong統一加在最後
            else:
                WrongWord[correct][wrong][0]+=1

In [9]:
from collections import OrderedDict
standardizeWW = OrderedDict()
n= 0 
for word in firstword:
    word_dict =WrongWord[word]
    if word_dict == {}:
        standardizeWW[word] = {}
        continue
    sorted_word_dict = sorted(word_dict.items(), key=lambda kv: (kv[1][0],kv[1][1]), reverse=True)
    standardizeWW [word] = OrderedDict()
    for s in sorted_word_dict:
        if s[1][0]>=10:
            n+=1
            standardizeWW [word][s[0]] = s[1][0]

# 全字庫
https://www.cns11643.gov.tw/
* 獲得字音、部件、筆畫資料

In [10]:
with open("data/CNS.txt", "rt",encoding="utf-8") as fin:
    txt = fin.readlines()
CNS={}
for t in txt:
    split = t[:-1].split(",")
    CNS[split[0]] = split[1] 

In [11]:
with open("data/CNS/one_phonetic.xml", "rt",encoding="utf-8") as fin:
    txt = fin.read()
soup = BeautifulSoup(txt, "lxml")
#統一拼音
one_phonetic = {}
for w in firstword:
    one_phonetic[w] = {}
for s in soup.select("word"):
    phone = s.find("phone").text.split("\u3000")
    word = list(s["item"])
    
    for w,p in zip(word,phone):
        if w not in one_phonetic.keys():
            continue
        else:
            if p not in one_phonetic[w].keys(): 
                one_phonetic[w][p] = 1
            else:
                one_phonetic[w][p] += 1

In [12]:
initials = ["ㄅ","ㄆ","ㄇ","ㄈ","ㄉ","ㄊ","ㄋ","ㄌ","ㄍ","ㄎ","ㄏ","ㄐ","ㄑ","ㄒ","ㄓ","ㄔ","ㄕ","ㄖ","ㄗ","ㄘ","ㄙ"]
finals = ["ㄚ","ㄛ","ㄜ","ㄝ","ㄟ","ㄞ","ㄠ","ㄡ","ㄢ","ㄤ","ㄣ","ㄥ","ㄦ"]
jiein = ["ㄧ","ㄨ","ㄩ"]
tone = ["˙","-","ˊ","ˇ","ˋ"]
with open("data/CNS/CNS_phonetic.txt", "rt",encoding="utf-8") as fin:
    txt = fin.readlines()
#拼音
phonetic={}
for t in txt:
    p = t[:-1].replace("\t","-",1).split("\t")
    yin = ["#","#","#","1"]
    for w in p[1]:
        if w in initials:
            yin[0] = w
        elif w in jiein:
            yin[1] = w
        elif w in finals:
            yin[2] = w    
        elif w in tone:
            yin[3] = str(tone.index(w))
    p[1] = "".join(yin)
    if p[0] in phonetic.keys():
        phonetic[p[0]].append(p[1])
    else:
        phonetic[p[0]] = []
        phonetic[p[0]].append(p[1])

In [13]:
with open("data/CNS/CNS_stroke.txt", "rt",encoding="utf-8") as fin:
    txt = fin.readlines()
#筆畫
stroke={}
for t in txt:
    c = t[:-1].replace("\t","-",1).split("\t")
    stroke[c[0]] = int(c[1])

In [14]:
with open("data/CNS/CNS_component.txt", "rt",encoding="utf-8") as fin:
    txt = fin.readlines()
#部件
component={}
for t in txt:
    c = t[:-1].replace("\t","-",1).split("\t")
    component[c[0]] = []
    if ";" in c[1]:
        another = c[1].split(";")
        for a in another:
            component[c[0]].append(a.split(","))
    else:
        component[c[0]].append(c[1].split(","))

In [15]:
CNS_word = {}
for f in firstword:
    CNS_word[f] = []
    #component
    CNS_word[f].append(component[CNS[f]])
    #phonetic
    if len(phonetic[CNS[f]])>1:
        try:
            p = max(one_phonetic[f], key=one_phonetic[f].get)
            yin = ["#","#","#","1"]
            for w in p:
                if w in initials:
                    yin[0] = w
                elif w in jiein:
                    yin[1] = w
                elif w in finals:
                    yin[2] = w    
                elif w in tone:
                    yin[3] = str(tone.index(w))
            p = "".join(yin)
            CNS_word[f].append(p)
            
        except:
            CNS_word[f].append(phonetic[CNS[f]][0])
    else:
        CNS_word[f].append(phonetic[CNS[f]][0])
    #stroke
    CNS_word[f].append(stroke[CNS[f]])

# 部件
* 順序：jaccard相似度>筆畫

In [16]:
def jaccard_sim(a, b):
    unions = len(set(a).union(set(b)))
    intersections = len(set(a).intersection(set(b)))
    return 1. * intersections / unions

In [17]:
jaccard = {} #詞表字與其他詞表字的jaccard相似度
for w in CNS_word:  
    jaccard[w] = {}
    for i in CNS_word:
        jaccard[w][i] = [0,abs(CNS_word[i][2]-CNS_word[w][2])*(-1)] #相似度(預留),筆畫差

In [18]:
for w in CNS_word:  #w:詞表字  i:對應的四千七 n:多個部件
    if len(CNS_word[w][0]) ==1: #部件組成方式僅有一種
        jac = CNS_word[w][0][0]
        for i in CNS_word:
            jaccard[w][i][0]=jaccard_sim(jac,CNS_word[i][0][0])
            
    else: #部件組成方式超過一種 取相似度最大的一種
        for jac_word in CNS_word:
            for fs1 in CNS_word[w][0]:
                for fs2 in CNS_word[jac_word][0]: 
                    jac = jaccard_sim(fs1,fs2)
                    if  jac > jaccard[w][jac_word][0]:
                        jaccard[w][jac_word][0] = jac
                    jaccard[jac_word][w][0] = jaccard[w][jac_word][0]

In [19]:
#刪去相似度小於0.5及本身字
for jac in jaccard:
    for j in firstword: 
        if jaccard[jac][j][0] < 0.5 or jac==j:
            del jaccard[jac][j]

In [20]:
from collections import OrderedDict
#排序
standardizeCP = OrderedDict()
for word in firstword:
    word_dict = jaccard[word]
    if word_dict == {}:
        standardizeCP[word] = {}
        continue
    sorted_word_dict = sorted(word_dict.items(), key=lambda kv: (kv[1][0],kv[1][1]), reverse=True)
    standardize_word = OrderedDict()
    for s in sorted_word_dict:
        standardize_word[s[0]] = s[1][0] 
    standardizeCP[word] = standardize_word

# 拼音
* 聲母+介音+韻母+聲調+常見程度

In [21]:
import math
def distance(a,b):
    d = 0
    for x,y in zip(a,b):
        d += (x-y)**2
    return math.sqrt(d)

In [22]:
init ={"ㄅ":(1,1),"ㄆ":(1,2),"ㄇ":(1,5),"ㄈ":(2,7),"ㄉ":(3,1),"ㄊ":(3,2),"ㄋ":(3,5),"ㄌ":(3,6),
 "ㄍ":(4,1),"ㄎ":(4,2),"ㄏ":(4,7),"ㄐ":(5,3),"ㄑ":(5,4),"ㄒ":(5,7),"ㄓ":(6,3),"ㄔ":(6,4),
 "ㄕ":(6,7),"ㄖ":(6,8),"ㄗ":(7,3),"ㄘ":(7,4),"ㄙ":(7,7)}

In [23]:
initials_distance={} #聲母距離
for p in init:
    initials_distance[p] = {} 
    for i in init:
        initials_distance[p][i] = distance(init[p],init[i])
    initials_distance[p]["#"] = 9
initials_distance["#"] = {}#零聲母
for p in init:
    initials_distance["#"][p] = 9
    initials_distance["#"]["#"] = 1
#標準化
for w in initials_distance:
    Max = max(initials_distance[w].values())
    Min = min(initials_distance[w].values())
    for i in initials_distance[w]:
        initials_distance[w][i] = 1-((initials_distance[w][i]-Min)/(Max-Min))

In [24]:
vowel={"i":(1,1,0),"y":(1,1,1),"u":(3,1,1),"e":(1,3,0),"ɤ":(3,3,0),"o":(3,3,1),"ə":(2,4,0),"a":(1,7,0),"ɑ":(3,7,0)}
fin = {'ㄚ':"a", 'ㄛ':"o", 'ㄜ':"ɤ", 'ㄝ':"e", 'ㄟ':"ei", 'ㄞ':"ai", 'ㄠ':"ɑu", 'ㄡ':"ou", 'ㄢ':"an", 'ㄤ':"ɑŋ", 'ㄣ':"ən", 'ㄥ':"əŋ", 'ㄦ':"ɚ"}

In [25]:
def count_finals_distance(A,B):
    dist=0
    n=0
    for a in fin[A]:
        for b in fin[B]:
            n+=1
            if A == B:
                dist = 0
                continue            
            if a in "ɚ" or b=="ɚ":
                dist += 7
            elif a in ("n","ŋ") and b not in ("n","ŋ"):
                dist += 7
            elif a not in ("n","ŋ") and b in ("n","ŋ"):
                dist += 7
            elif a in ("n","ŋ") and b in ("n","ŋ"):
                if a == b:
                    dist +=0
                else:
                    dist += 1
            else:
                dist += distance(vowel[a],vowel[b])
    return dist/n

In [26]:
finals_distance={}#韻母發音距離
for f in fin:
    finals_distance[f] = {} 
    for i in fin:
        finals_distance[f][i] =count_finals_distance(f,i)
    finals_distance[f]["#"] = 7
finals_distance["#"] = {}
for f in fin:
    finals_distance["#"][f] = 7
    finals_distance["#"]["#"] = 1
#標準化
for w in finals_distance:
    Max = max(finals_distance[w].values())
    Min = min(finals_distance[w].values())
    for i in finals_distance[w]:
        finals_distance[w][i] = 1-((finals_distance[w][i]-Min)/(Max-Min))

In [27]:
Phonetic = {}
L = len(firstword)
for w in firstword:
    firstword_phonetic = CNS_word[w][1]
    Phonetic[w] = {}
    for i in firstword:
        if w == i:
            continue
        word_phonetic = CNS_word[i][1]
        I = initials_distance[firstword_phonetic[0]][word_phonetic[0]] #聲母
        if firstword_phonetic[1] == word_phonetic[1]: #介音
            J = 1
        else:
            J = 0
        F = finals_distance[firstword_phonetic[2]][word_phonetic[2]] # 韻母
        T = 1 - abs(int(firstword_phonetic[3])- int(word_phonetic[3]))/4 #聲調
        O = (1-firstword.index(i)/L)/2
        Phonetic[w][i] = (I+J+F+T+O)/4.5

In [28]:
from collections import OrderedDict
standardizePH = OrderedDict()
for f in firstword:
    standardizePH[f] = OrderedDict()
    sorted_dict = sorted(Phonetic[f].items(), key=lambda kv: kv[1], reverse=True)
    for s in sorted_dict[:10]:
        standardizePH[f][s[0]] = s[1]

# Final

In [29]:
Final = {}
for f in firstword:
    Final[f] = {}
    for w in standardizeWW[f]:
        Final[f][w] = standardizeWW[f][w]
    for c in standardizeCP[f]:
        if c in Final[f].keys():
            Final[f][c] += standardizeCP[f][c]
        else:
            Final[f][c] = standardizeCP[f][c]
    for p in standardizePH[f]:
        if p in Final[f].keys():
            Final[f][p] += standardizePH[f][p]
        else:
            Final[f][p] = standardizePH[f][p]

In [30]:
with open("ConfusionSet.txt", "wt",encoding="utf-8") as fin:
    for f in Final:
        word_dict =Final[f]
        sorted_word_dict = sorted(word_dict.items(), key=lambda kv: kv[1], reverse=True)
        sorted_word=""
        for s in sorted_word_dict[:10]:
            sorted_word +=s[0]
        fin.write(f+","+sorted_word+"\n")